In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#filepath = 'C:/Users/I008328/Desktop/AIA/Next Recommended Actions/'
filepath = 'D:/Users/I008328/NRA/data/'
plt.style.use('seaborn')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.3f}'.format)

In [2]:
import os
import pickle
import pyodbc
import cx_Oracle
import sqlalchemy
import time
import datetime
import re
from sqlalchemy import create_engine

In [3]:
start = time.time()
 
#oracle_connection_string = 'oracle+cx_oracle://{username}:{password}@{hostname}:{port}/?service_name={database}'
oracle_connection_string = 'oracle+cx_oracle://{username}:{password}@{host}:{port}/?service_name={database}'

 

engine = create_engine(
    oracle_connection_string.format(
        username='KMDB016',
        password='mktBAtg1688!',
#        hostname='mybfdp1c1-scan.my1.ocm.s7067199.oraclecloudatcustomer.com',
        host = '10.136.107.7',
        #hostname = 'KUDWH01.my2.ocm.s7067200.oraclecloudatcustomer.com',
        port='1521',
        database='KPDWH01.MY1.OCM.S7067199.ORACLECLOUDATCUSTOMER.COM'
    )
    )

In [4]:
# Only Get Individual Customers
#sql  = " SELECT * FROM NRA_PRED_SEP_TRAIN_2020 "

sql = " select * from pred_sep_2020_temp "

df_sep = pd.read_sql(sql, engine)

end = time.time()

print("Time Taken: {:.2f}s".format(end-start))

df_sep.shape

Time Taken: 106.21s


(2384841, 3)

In [5]:
df_sep.to_pickle(filepath + 'Sep2020_Pred.pkl')

In [6]:
sql = " SELECT * FROM INS_NB_4M_OCT_2020 "
df_repurchase = pd.read_sql(sql, engine)
df_repurchase.shape

(149731, 3)

In [44]:
df_repurchase.to_pickle(filepath + 'Oct_2020_Repurchase.pkl')

In [5]:
df_sep_flags = pd.read_pickle(filepath + 'Sep2020_snapshot.pkl')
#df_sep = pd.read_pickle(filepath + 'Sep2020_Pred.pkl')
df_repurchase = pd.read_pickle(filepath + 'Oct_2020_Repurchase.pkl')

In [4]:
df_sep.columns = df_sep.columns.str.upper()

In [5]:
df_sep.drop_duplicates(subset='INS_ID',keep='first',inplace=True, ignore_index=True)
df_sep.shape

(2384841, 58)

In [8]:
df_sep_flags.columns

Index(['REF_DT', 'CUSTOMER_UNIQUE_ID', 'COMMON_IC', 'NEW_IC', 'OLD_IC', 'SERVICING_AGENT_CD', 'SCV_ID', 'CUSTOMER_ROLE', 'INFORCE_IND', 'ANN_INCOME', 'MARITAL_STATUS', 'LIFE_STAGE', 'OCCUPATION', 'NO_OF_CHILD', 'VIT_MEMBERSHIP_STATUS', 'VIT_STATUS_LAST_12M', 'VIT_AMOUNT_DUE', 'VIT_AMOUNT_DUE_DATE', 'VIT_POINTS', 'VIT_STATUS_CURRENT', 'VIT_POITNS_BAL', 'VIT_BAL_DUE_DATE', 'VIT_STATUS_BAL', 'VIT_BAL_STATEMENT_IND', 'FLG_REPURCHASE_YTD', 'REPURCHASE_LIKELIHOOD', 'LAPSE_LIKELIHOOD', 'AD_CONV_LIKELIHOOD', 'FLG_AGE_21', 'FLG_MARRIED', 'FLG_MARRIED_UPDATE_DT', 'FLG_LIFE_STAGE', 'FLG_LIFE_STAGE_UPDATE_DT', 'FLG_CHILD', 'FLG_CHILD_UPDATE_DT', 'FLG_INCOME_CHANGE', 'FLG_INCOME_UPDATE_DT', 'FLG_BIRTHDAY', 'FLG_POL_ANNIVERSARY', 'FLG_FHC_ANNUAL', 'FLG_NON_ACUTE_CLM', 'FLG_NON_ACUTE_CLM_AMT', 'FLG_ACUTE_CLM', 'FLG_ACUTE_CLM_AMT', 'FLG_CLM_TRIGGER', 'FLG_ACC_CLM', 'FLG_ACC_CLM_AMT', 'FLG_NON_ACUTE_CLM_FAM', 'FLG_NON_ACUTE_CLM_AMT_FAM', 'FLG_ACUTE_CLM_FAM', 'FLG_ACUTE_CLM_AMT_FAM', 'FLG_ACC_CLM_FAM',


In [10]:
#df_train = df_sep.merge(df_sep_flags, on=['SCV_ID'],suffixes=('_pred', '_flags'))
#df_train.shape

(3530233, 124)

In [6]:
df_flags = df_sep_flags[['SCV_ID','FLG_AGE_21','FLG_MARRIED','FLG_LIFE_STAGE','FLG_CHILD','FLG_INCOME_CHANGE','FLG_BIRTHDAY','FLG_FHC_ANNUAL','FLG_CLM_TRIGGER',
'FLG_CLM_FAM_TRIGGER','FLG_CALL_TRIGGER','FLG_MATURING','FLG_MATURED']]
df_flags.head()

SCV_ID FLG_AGE_21 FLG_MARRIED FLG_LIFE_STAGE FLG_CHILD FLG_INCOME_CHANGE FLG_BIRTHDAY FLG_FHC_ANNUAL FLG_CLM_TRIGGER FLG_CLM_FAM_TRIGGER FLG_CALL_TRIGGER FLG_MATURING FLG_MATURED
0  14939955          N           N              N         N                 N            N              N               N                   N                N            N           N
1   3407282          N           N              N         N                 N            N              N               N                   N                N            N           N
2   2696849          N           N              N         N                 N            N              N               N                   N                N            N           N
3   2696849          N           N              N         N                 N            N              N               N                   N                N            N           N
4  12394011          N           N              N         N                 N            N              N               N                   N                N            N           N

In [7]:
targets = {'Y':1, 'N':0}
df_flags[['FLG_AGE_21','FLG_MARRIED','FLG_LIFE_STAGE','FLG_CHILD','FLG_INCOME_CHANGE','FLG_BIRTHDAY','FLG_FHC_ANNUAL','FLG_CLM_TRIGGER',
'FLG_CLM_FAM_TRIGGER','FLG_CALL_TRIGGER','FLG_MATURING','FLG_MATURED']] = \
df_flags_map = df_sep_flags[['FLG_AGE_21','FLG_MARRIED','FLG_LIFE_STAGE','FLG_CHILD','FLG_INCOME_CHANGE','FLG_BIRTHDAY','FLG_FHC_ANNUAL','FLG_CLM_TRIGGER',
'FLG_CLM_FAM_TRIGGER','FLG_CALL_TRIGGER','FLG_MATURING','FLG_MATURED']].stack().map(targets).unstack()

C:\Users\I008328\.conda\envs\kw-py37\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [8]:
df_flags1 = df_flags.groupby(['SCV_ID'],as_index=False).sum()
df_flags1

SCV_ID  FLG_AGE_21  FLG_MARRIED  FLG_LIFE_STAGE  FLG_CHILD  FLG_INCOME_CHANGE  FLG_BIRTHDAY  FLG_FHC_ANNUAL  FLG_CLM_TRIGGER  FLG_CLM_FAM_TRIGGER  FLG_CALL_TRIGGER  FLG_MATURING  FLG_MATURED
0               2           0            0               0          0                  0             0               0                0                    0                 0             0            0
1               3           0            0               1          0                  0             0               0                0                    0                 0             0            0
2               4           0            0               0          0                  0             4               0                0                    0                 0             0            0
3               5           0            0               0          0                  0             0               0                0                    0                 0             0            0
4               6           0            0               0          0                  0             0               0                0                    0                 0             0            0
...           ...         ...          ...             ...        ...                ...           ...             ...              ...                  ...               ...           ...          ...
2617456  20975571           0            0               0          0                  0             0               0                0                    0                 0             0            0
2617457  20975692           0            0               0          0                  0             0               0                0                    0                 0             0            0
2617458  20975734           0            0               0          0                  0             0               0                0                    0                 0             0            0
2617459  20975753           0            0               0          0                  0             0               0                0                    0                 0             0            0
2617460  20975865           0            0               0          0                  0             0               0                0                    0                 0             0            0

[2617461 rows x 13 columns]

In [9]:
# transform back to 0 and 1
df_flags1.iloc[:,1:][df_flags1.iloc[:,1:] > 0] = 1

C:\Users\I008328\.conda\envs\kw-py37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\I008328\.conda\envs\kw-py37\lib\site-packages\pandas\core\frame.py:3215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


In [10]:
df_flags1['FLG_SUM'] = df_flags1.iloc[:,1:].sum(axis=1)

In [11]:
cols = list(df_flags1.iloc[:,1:-1].columns)
cols

['FLG_AGE_21',
 'FLG_MARRIED',
 'FLG_LIFE_STAGE',
 'FLG_CHILD',
 'FLG_INCOME_CHANGE',
 'FLG_BIRTHDAY',
 'FLG_FHC_ANNUAL',
 'FLG_CLM_TRIGGER',
 'FLG_CLM_FAM_TRIGGER',
 'FLG_CALL_TRIGGER',
 'FLG_MATURING',
 'FLG_MATURED']

In [17]:
df_sep_flags.columns

Index(['REF_DT', 'CUSTOMER_UNIQUE_ID', 'COMMON_IC', 'NEW_IC', 'OLD_IC', 'SERVICING_AGENT_CD', 'SCV_ID', 'CUSTOMER_ROLE', 'INFORCE_IND', 'ANN_INCOME', 'MARITAL_STATUS', 'LIFE_STAGE', 'OCCUPATION', 'NO_OF_CHILD', 'VIT_MEMBERSHIP_STATUS', 'VIT_STATUS_LAST_12M', 'VIT_AMOUNT_DUE', 'VIT_AMOUNT_DUE_DATE', 'VIT_POINTS', 'VIT_STATUS_CURRENT', 'VIT_POITNS_BAL', 'VIT_BAL_DUE_DATE', 'VIT_STATUS_BAL', 'VIT_BAL_STATEMENT_IND', 'FLG_REPURCHASE_YTD', 'REPURCHASE_LIKELIHOOD', 'LAPSE_LIKELIHOOD', 'AD_CONV_LIKELIHOOD', 'FLG_MARRIED_UPDATE_DT', 'FLG_LIFE_STAGE_UPDATE_DT', 'FLG_CHILD_UPDATE_DT', 'FLG_INCOME_UPDATE_DT', 'FLG_POL_ANNIVERSARY', 'FLG_NON_ACUTE_CLM', 'FLG_NON_ACUTE_CLM_AMT', 'FLG_ACUTE_CLM', 'FLG_ACUTE_CLM_AMT', 'FLG_ACC_CLM', 'FLG_ACC_CLM_AMT', 'FLG_NON_ACUTE_CLM_FAM', 'FLG_NON_ACUTE_CLM_AMT_FAM', 'FLG_ACUTE_CLM_FAM', 'FLG_ACUTE_CLM_AMT_FAM', 'FLG_ACC_CLM_FAM', 'FLG_ACC_CLM_AMT_FAM', 'FLG_CALL_POSITIVE', 'CALL_DATE', 'CALL_TYPE', 'FLG_CALL_NEGATIVE', 'MYAIA_REGISTRATION_IND',
       'FHC_COMPL

In [12]:
df_sep_flags.drop(cols,axis=1,inplace=True)

In [13]:
df_sep_flags.shape

(3893318, 55)

In [14]:
df_sep_flags.drop_duplicates(subset='SCV_ID',keep='first',inplace=True, ignore_index=True)
df_sep_flags.shape

(2617461, 55)

In [21]:
df_flags1.iloc[:,1:-1]

FLG_AGE_21  FLG_MARRIED  FLG_LIFE_STAGE  FLG_CHILD  FLG_INCOME_CHANGE  FLG_BIRTHDAY  FLG_FHC_ANNUAL  FLG_CLM_TRIGGER  FLG_CLM_FAM_TRIGGER  FLG_CALL_TRIGGER  FLG_MATURING  FLG_MATURED
0                 0            0               0          0                  0             0               0                0                    0                 0             0            0
1                 0            0               1          0                  0             0               0                0                    0                 0             0            0
2                 0            0               0          0                  0             1               0                0                    0                 0             0            0
3                 0            0               0          0                  0             0               0                0                    0                 0             0            0
4                 0            0               0          0                  0             0               0                0                    0                 0             0            0
...             ...          ...             ...        ...                ...           ...             ...              ...                  ...               ...           ...          ...
2617456           0            0               0          0                  0             0               0                0                    0                 0             0            0
2617457           0            0               0          0                  0             0               0                0                    0                 0             0            0
2617458           0            0               0          0                  0             0               0                0                    0                 0             0            0
2617459           0            0               0          0                  0             0               0                0                    0                 0             0            0
2617460           0            0               0          0                  0             0               0                0                    0                 0             0            0

[2617461 rows x 12 columns]

In [15]:
# Sanity check to make sure only 0 and 1
df_flags1.iloc[:,1:-1].apply(pd.Series.value_counts).fillna(0)

FLG_AGE_21  FLG_MARRIED  FLG_LIFE_STAGE  FLG_CHILD  FLG_INCOME_CHANGE  FLG_BIRTHDAY  FLG_FHC_ANNUAL  FLG_CLM_TRIGGER  FLG_CLM_FAM_TRIGGER  FLG_CALL_TRIGGER  FLG_MATURING  FLG_MATURED
0     2609982      2603590         2478045    2580573            2599096       2377627     2617461.000          2613285              2602463           2582293       2606973      2613487
1        7479        13871          139416      36888              18365        239834           0.000             4176                14998             35168         10488         3974

In [16]:
df_train1 = pd.merge(df_sep_flags,df_flags1,how='inner',on='SCV_ID')
df_train1.shape


(2617461, 68)

In [18]:
#df_train2 = pd.merge(df_train1,df_repurchase,how='left',on='SCV_ID')
#
df_train2 = pd.merge(df_train1,df_repurchase,how='left',left_on='SCV_ID',right_on='SCV_ID')
df_train2.shape

(2617461, 70)

In [17]:
df_train1['SCV_ID'].isin(df_repurchase['SCV_ID']).value_counts()

False    2559493
True       57968
Name: SCV_ID, dtype: int64

In [19]:
df_train2['SCV_ID'].isin(df_repurchase['SCV_ID']).value_counts()

False    2559493
True       57968
Name: SCV_ID, dtype: int64

In [20]:
#Retrieve All flags
df_train2.loc[df_train2.FLG_SUM >0].shape[0]

462697

In [21]:
df_sep.shape

(2384841, 58)

In [26]:
df_sep['SCV_ID'].isin(df_train2['SCV_ID']).value_counts()

True     2347762
False      37079
Name: SCV_ID, dtype: int64

In [23]:
#missing
2384841 - 2347762

37079

In [30]:
#Retrieve flag with repurchase
df_train2.loc[(df_train2.TARGET_REPUR_IND ==1) & (df_train2.FLG_SUM >0)].shape[0]

14992

In [33]:
#df_train = pd.read_csv(filepath + 'df_sep_flags.csv')
#df_train.shape

(2617461, 16)

In [34]:
#Retrieve flag with repurchase
df_train.loc[(df_train.TARGET_REPUR_IND ==1) & (df_train.FLG_SUM >0)].shape[0]

14992

In [ ]:
#df_train1 = df_sep.merge(df_sep_flags, left_on='SCV_ID', right_on='SCV_ID',suffixes=('_pred', '_flag'))

In [27]:
df_train3a = df_train2.merge(df_sep, left_on='SCV_ID', right_on='SCV_ID',suffixes=('_flag', '_pred'))
df_train3a.shape

(2347762, 127)

In [24]:
df_train3 = df_train2.join(df_sep, on='SCV_ID', how='left', lsuffix='_flag', rsuffix='_pred')
df_train3.shape

(2617461, 128)

In [28]:
df_train3[['SCV_ID_flag','SCV_ID_pred']]

SCV_ID_flag  SCV_ID_pred
0           14939955          NaN
1            3407282          NaN
2            2696849          NaN
3           12394011          NaN
4            5804575          NaN
...              ...          ...
2617456     18618158          NaN
2617457      2590690          NaN
2617458      3046280          NaN
2617459     14376593          NaN
2617460      2055644 19085345.000

[2617461 rows x 2 columns]

In [25]:
df_train3.SCV_ID_pred.nunique()

747357

In [32]:
2617461 -df_train3a.SCV_ID.nunique()

269699

In [31]:
df_train3a.head()

REF_DT_flag     CUSTOMER_UNIQUE_ID     COMMON_IC        NEW_IC OLD_IC SERVICING_AGENT_CD    SCV_ID CUSTOMER_ROLE INFORCE_IND          ANN_INCOME MARITAL_STATUS          LIFE_STAGE                      OCCUPATION  NO_OF_CHILD VIT_MEMBERSHIP_STATUS VIT_STATUS_LAST_12M  VIT_AMOUNT_DUE VIT_AMOUNT_DUE_DATE  VIT_POINTS VIT_STATUS_CURRENT  VIT_POITNS_BAL VIT_BAL_DUE_DATE VIT_STATUS_BAL VIT_BAL_STATEMENT_IND FLG_REPURCHASE_YTD REPURCHASE_LIKELIHOOD LAPSE_LIKELIHOOD AD_CONV_LIKELIHOOD FLG_MARRIED_UPDATE_DT FLG_LIFE_STAGE_UPDATE_DT FLG_CHILD_UPDATE_DT FLG_INCOME_UPDATE_DT FLG_POL_ANNIVERSARY FLG_NON_ACUTE_CLM  FLG_NON_ACUTE_CLM_AMT FLG_ACUTE_CLM  FLG_ACUTE_CLM_AMT FLG_ACC_CLM  FLG_ACC_CLM_AMT FLG_NON_ACUTE_CLM_FAM  FLG_NON_ACUTE_CLM_AMT_FAM FLG_ACUTE_CLM_FAM  FLG_ACUTE_CLM_AMT_FAM FLG_ACC_CLM_FAM  FLG_ACC_CLM_AMT_FAM FLG_CALL_POSITIVE CALL_DATE CALL_TYPE FLG_CALL_NEGATIVE MYAIA_REGISTRATION_IND  ... INS_GENDER INS_MARITAL_STATUS      INS_LIFE_STAGE                   INS_OCCUPATON  \
0  2020-09-30  013187372455-19780831  013187372455  013187372455   None              4854Q  14939955          BOTH           Y    RM 36,001-72,000         Single       MATURE SINGLE   Trading (Office Work - Import            0                  None                None             NaN                 NaT         NaN               None             NaN              NaT           None                  None                  N                     -  Somewhat Likely                  -                   NaT                      NaT                 NaT                  NaT                   N                 N                  0.000             N              0.000           N            0.000                     N                      0.000                 N                  0.000               N                0.000                 N       NaT      None                 N                      N  ...          F             Single       Mature Single   Trading (Office Work - Import   
1  2020-09-30  013187372483-19780831  013187372483  013187372483   None              A7242   3407282            LF           Y    RM 36,001-72,000         Single       MATURE SINGLE  Senior Management, Manager, Ex            0                  None                None             NaN                 NaT         NaN               None             NaN              NaT           None                  None                  N                     -  Somewhat Likely                  -                   NaT                      NaT                 NaT                  NaT                   N                 N                  0.000             N              0.000           N            0.000                     N                      0.000                 N                  0.000               N                0.000                 N       NaT      None                 N                      Y  ...          F             Single       Mature Single  Senior Management, Manager, Ex   
2  2020-09-30  013187372574-19780831  013187372574  013187372574   None              1363A   2696849          BOTH           Y  RM 180,001-300,000        Married        NEST BUILDER                  Sales Director            1                  None                None             NaN                 NaT         NaN               None             NaN              NaT           None                  None                  N                     -  Somewhat Likely                  -                   NaT                      NaT                 NaT                  NaT                   Y                 N                  0.000             N              0.000           N            0.000                     N                      0.000                 N                  0.000               N                0.000                 N       NaT      None                 N                      N  ...          M            Married        Nest Builder                  Sales Director   
3  2020-09-30  013187372712-19780831  0131

In [55]:
##################

df_train4 = df_train3a[df_train3a.SCV_ID.notna()]
df_train4.shape

(2347762, 127)

In [56]:
df_train4.loc[(df_train4.TARGET_REPUR_IND ==1) & (df_train4.FLG_SUM >0)].shape[0]

14360

In [61]:
df_train4.dtypes

REF_DT_flag                  datetime64[ns]
CUSTOMER_UNIQUE_ID                   object
COMMON_IC                            object
NEW_IC                               object
OLD_IC                               object
SERVICING_AGENT_CD                   object
SCV_ID                                int64
CUSTOMER_ROLE                        object
INFORCE_IND                          object
ANN_INCOME                           object
MARITAL_STATUS                       object
LIFE_STAGE                           object
OCCUPATION                           object
NO_OF_CHILD                           int64
VIT_MEMBERSHIP_STATUS                object
VIT_STATUS_LAST_12M                  object
VIT_AMOUNT_DUE                      float64
VIT_AMOUNT_DUE_DATE          datetime64[ns]
VIT_POINTS                          float64
VIT_STATUS_CURRENT                   object
VIT_POITNS_BAL                      float64
VIT_BAL_DUE_DATE             datetime64[ns]
VIT_STATUS_BAL                  

In [ ]:
MARITAL_STATUS                       object
LIFE_STAGE                           object
OCCUPATION                           object
NO_OF_CHILD                           int64
VIT_MEMBERSHIP_STATUS                object
VIT_STATUS_LAST_12M  

In [63]:
df_train4.isnull().sum().sort_values(ascending=False).head(50)

VIT_HEALTHY_FOOD_MTH        2347762
VIT_STATUS                  2347762
VIT_FITNESS_MTH             2347762
VIT_HEALTH_CHK              2347762
VIT_ENGAGEMENT_3M           2347762
TOP3_FLAG                   2342030
FLG_MARRIED_UPDATE_DT       2337521
FLG_INCOME_UPDATE_DT        2334048
VIT_STATUS_LAST_12M         2322566
FLG_CHILD_UPDATE_DT         2316782
CALL_TYPE                   2315540
CALL_DATE                   2315540
REPURCHASE_ANP              2293560
TARGET_REPUR_IND            2293560
VIT_AMOUNT_DUE_DATE         2269907
TOP2_FLAG                   2268568
VIT_POINTS                  2268089
VIT_AMOUNT_DUE              2267469
VIT_BAL_DUE_DATE            2267370
VIT_BAL_STATEMENT_IND       2267370
VIT_STATUS_BAL              2267370
VIT_POITNS_BAL              2267370
VIT_STATUS_CURRENT          2267370
FLG_LIFE_STAGE_UPDATE_DT    2234596
VIT_MEMBERSHIP_STATUS       2202576
OLD_IC                      2026184
INS_INCOME_DT               1412983
INS_INCOME                  

In [57]:
df_train4.INS_AGE.isnull().sum()

97

In [64]:
df_train4['REPURCHASE_ANP'] = df_train4['REPURCHASE_ANP'].fillna(0)
df_train4['TARGET_REPUR_IND'] = df_train4['TARGET_REPUR_IND'].fillna(0)

In [65]:
# Use columns that don't have missing values
df_train5 = df_train4[df_train4.columns[~df_train4.isnull().any()]]
df_train5.columns

Index(['REF_DT_flag', 'CUSTOMER_UNIQUE_ID', 'COMMON_IC', 'SERVICING_AGENT_CD', 'SCV_ID', 'CUSTOMER_ROLE', 'INFORCE_IND', 'NO_OF_CHILD', 'FLG_REPURCHASE_YTD', 'REPURCHASE_LIKELIHOOD', 'LAPSE_LIKELIHOOD', 'AD_CONV_LIKELIHOOD', 'FLG_POL_ANNIVERSARY', 'FLG_NON_ACUTE_CLM', 'FLG_NON_ACUTE_CLM_AMT', 'FLG_ACUTE_CLM', 'FLG_ACUTE_CLM_AMT', 'FLG_ACC_CLM', 'FLG_ACC_CLM_AMT', 'FLG_NON_ACUTE_CLM_FAM', 'FLG_NON_ACUTE_CLM_AMT_FAM', 'FLG_ACUTE_CLM_FAM', 'FLG_ACUTE_CLM_AMT_FAM', 'FLG_ACC_CLM_FAM', 'FLG_ACC_CLM_AMT_FAM', 'FLG_CALL_POSITIVE', 'FLG_CALL_NEGATIVE', 'MYAIA_REGISTRATION_IND', 'FHC_COMPLETION_IND', 'FLG_PPT', 'FLG_AGE_21', 'FLG_MARRIED', 'FLG_LIFE_STAGE', 'FLG_CHILD', 'FLG_INCOME_CHANGE', 'FLG_BIRTHDAY', 'FLG_FHC_ANNUAL', 'FLG_CLM_TRIGGER', 'FLG_CLM_FAM_TRIGGER', 'FLG_CALL_TRIGGER', 'FLG_MATURING', 'FLG_MATURED', 'FLG_SUM', 'REPURCHASE_ANP', 'TARGET_REPUR_IND', 'INS_ID', 'REF_DT_pred', 'INS_INCOME_SEGM', 'INS_GENDER', 'INS_NON_ACUTE', 'INS_CLM_MAJOR', 'INS_ACUTE_CT_6', 'INS_PAID_AMT_ACUTE_6',


In [67]:
df_train5 = pd.concat([df_train5,df_train4[['INS_L_AGT_SEGM','INS_AGE','INS_LIFE_STAGE']]],axis=1)

In [68]:
df_train5[['FLG_AGE_21','FLG_MARRIED','FLG_LIFE_STAGE','FLG_CHILD','FLG_INCOME_CHANGE','FLG_BIRTHDAY','FLG_FHC_ANNUAL','FLG_CLM_TRIGGER',
'FLG_CLM_FAM_TRIGGER','FLG_CALL_TRIGGER','FLG_MATURING','FLG_MATURED']].apply(pd.Series.value_counts).fillna(0)

FLG_AGE_21  FLG_MARRIED  FLG_LIFE_STAGE  FLG_CHILD  FLG_INCOME_CHANGE  FLG_BIRTHDAY  FLG_FHC_ANNUAL  FLG_CLM_TRIGGER  FLG_CLM_FAM_TRIGGER  FLG_CALL_TRIGGER  FLG_MATURING  FLG_MATURED
0     2340607      2334494         2215514    2313669            2329885       2131095     2347762.000          2343632              2333364           2314955       2337730      2344058
1        7155        13268          132248      34093              17877        216667           0.000             4130                14398             32807         10032         3704

In [69]:
df_train5 = df_train5[df_train5.FLG_SUM > 0]
df_train5.shape

(426915, 86)

In [70]:
df_train5.FLG_SUM.value_counts()

1    377122
2     41800
3      6477
4      1359
5       152
6         5
Name: FLG_SUM, dtype: int64

In [71]:
df_train6 = df_train5.drop(df_train5[(df_train5.FLG_SUM == 1) & (df_train5.FLG_BIRTHDAY == 1) &
                                     (df_train5.TARGET_REPUR_IND == 0)].index)
df_train6.shape

(239194, 86)

In [72]:
df_train7 = df_train6.drop(df_train6[(df_train6.FLG_SUM == 1) & (df_train6.FLG_LIFE_STAGE == 1) &
                                     (df_train6.TARGET_REPUR_IND == 0)].index)
df_train7.shape

(144734, 86)

In [73]:
df_train7[['FLG_AGE_21','FLG_MARRIED','FLG_LIFE_STAGE','FLG_CHILD','FLG_INCOME_CHANGE','FLG_BIRTHDAY','FLG_FHC_ANNUAL','FLG_CLM_TRIGGER',
'FLG_CLM_FAM_TRIGGER','FLG_CALL_TRIGGER','FLG_MATURING','FLG_MATURED']].apply(pd.Series.value_counts).fillna(0)

FLG_AGE_21  FLG_MARRIED  FLG_LIFE_STAGE  FLG_CHILD  FLG_INCOME_CHANGE  FLG_BIRTHDAY  FLG_FHC_ANNUAL  FLG_CLM_TRIGGER  FLG_CLM_FAM_TRIGGER  FLG_CALL_TRIGGER  FLG_MATURING  FLG_MATURED
0      137579       131466          106946     110641             126857        115788      144734.000           140604               130336            111927        134702       141030
1        7155        13268           37788      34093              17877         28946           0.000             4130                14398             32807         10032         3704

In [74]:
df_train7.NO_OF_CHILD.value_counts()

0     70292
1     25181
2     22712
3     15453
4      7925
5      2103
6       702
7       219
8        76
9        30
10       22
11        9
12        5
13        3
15        1
16        1
Name: NO_OF_CHILD, dtype: int64

In [75]:
df_train7.to_csv(filepath + 'cust360_sep_cleaned.csv',index=False)